In [1]:
import tensorflow as tf

In [95]:
path = '../../data/preprocessed_data/test'
data = tf.data.Dataset.load(path)

/Users/andreeaioanatudor/Desktop/Uni/Msc/HWR/Handwriting-Recognition/english_handwriting/venv/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7173451646684226902


In [87]:
if tf.config.run_functions_eagerly(True):
    print("Eager execution is enabled.")
else:
    print("Eager execution is not enabled.")

Eager execution is not enabled.


In [92]:
tf.config.run_functions_eagerly(True)

In [96]:



ds = data.take(1)
for item in ds:
    img = item['image']
    print(type(img))
    array = img.numpy()
    print(array)

<class 'tensorflow.python.framework.ops.EagerTensor'>
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7173451646684226902


In [21]:
import matplotlib.pyplot as plt
from keras.src.layers import StringLookup
import numpy as np

In [19]:
train_path = '../../data/preprocessed_data/train'
vocab_path = '../../data/preprocessed_data/vocab.txt'

In [57]:
decoder_vocab = []
with open(vocab_path, 'r') as f:
    for line in f:
        decoder_vocab.append(line.strip())



train_ds = tf.data.Dataset.load(train_path)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17768592637663142582


In [112]:
import random
import cv2
import numpy as np
import albumentations as A
import tensorflow as tf
from PIL import Image


# gets PIL image and returns augmented PIL image
def augment_img(img: tf.Tensor):
    # only augment 3/4th the images
    if random.randint(1, 4) > 3:
        return img
    print(type(img))
    img = tf.make_ndarray(img)
    # img = img.numpy()
    # img = np.asarray(img)  # convert to numpy for opencv
    print(img.dtype)

    # morphological alterations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    if random.randint(1, 5) == 1:
        # dilation because the image is not inverted
        img = cv2.erode(img, kernel, iterations=random.randint(1, 2))
    if random.randint(1, 6) == 1:
        # erosion because the image is not inverted
        img = cv2.dilate(img, kernel, iterations=random.randint(1, 1))

    img = Image.fromarray(img)

    # noise introduction
    transform = A.Compose([

        A.OneOf([
            # add black pixels noise
            A.OneOf([
                # A.RandomRain(brightness_coefficient=1.0, drop_length=2, drop_width=2, drop_color = (0, 0, 0), blur_value=1, rain_type = 'drizzle', p=0.05),
                #  A.RandomShadow(p=1),
                A.PixelDropout(p=1),
            ], p=0.9),

            # add white pixels noise
            A.OneOf([
                A.RandomRain(brightness_coefficient=1.0, drop_length=2,
                             drop_width=2, drop_color=(255, 255, 255),
                             blur_value=1, rain_type=None, p=1),
            ], p=0.9),
        ], p=1),

        # transformations
        A.OneOf([
            A.ShiftScaleRotate(shift_limit=0, scale_limit=0.25, rotate_limit=2,
                               border_mode=cv2.BORDER_CONSTANT,
                               value=(255, 255, 255), p=1),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0, rotate_limit=8,
                               border_mode=cv2.BORDER_CONSTANT,
                               value=(255, 255, 255), p=1),
            A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.15,
                               rotate_limit=11, border_mode=cv2.BORDER_CONSTANT,
                               value=(255, 255, 255), p=1),
            A.Affine(shear=random.randint(-5, 5), mode=cv2.BORDER_CONSTANT,
                     cval=(255, 255, 255), p=1),
            A.ElasticTransform(alpha=random.randint(0, 10),
                               sigma=random.randint(0, 100)),
        ], p=0.5),
        # A.Blur(blur_limit=5,p=0.25),
    ])
    img = transform(image=img)

    img = tf.convert_to_tensor(img)

    return img


# def augment_using_ops(images, labels):
# 	# randomly flip the images horizontally, randomly flip the images
# 	# vertically, and rotate the images by 90 degrees in the counter
# 	# clockwise direction
# 	images = tf.image.random_flip_left_right(images)
# 	images = tf.image.random_flip_up_down(images)
# 	images = tf.image.rot90(images)
# 	# return the image and the label
# 	return (images, labels)

# def load_batch():
#     ds = tf.data.Dataset.from_tensor_slices(imagePaths)
#     ds = (ds
#           # .shuffle(len(imagePaths), seed=42)
#           # .map(load_images, num_parallel_calls=AUTOTUNE)
#           # .cache()
#           # .batch(BATCH_SIZE)
#           # call the augmentation method here
#           .map(augment_img, num_parallel_calls=AUTOTUNE)
#           # .prefetch(tf.data.AUTOTUNE)
#           )
#
#     return ds





In [113]:
from util.global_params import *

# from data_augmentation import augment_img

In [114]:
ds = train_ds.take(1)
print(type(ds))
for item in ds:
    img = item['image']
    print(type(img))
    array = img.numpy()

<class 'tensorflow.python.data.ops.take_op._TakeDataset'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17768592637663142582
2023-06-10 22:48:11.552777: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [115]:
tf.config.run_functions_eagerly(True)

train_ds = tf.data.Dataset.load(train_path)
train_ds = train_ds.batch(BATCH_SIZE).cache().prefetch(
        buffer_size=AUTOTUNE)

# Calculate the length of the batch before augmentation
# batch_length_before = tf.data.experimental.cardinality(
#     train_ds).numpy()
# print("Batch Length Before Augmentation:", batch_length_before)

# augment the data
augmented_images_batches = train_ds.map(
    lambda x: {'image': augment_img(x['image']), 'label': x['label']},
    num_parallel_calls=AUTOTUNE)
# train_batches = train_ds.concatenate(augmented_images_batches)
# # Calculate the length of the batch after augmentation
# batch_length_after = tf.data.experimental.cardinality(
#     train_batches).numpy() * BATCH_SIZE
# print("Batch Length After Augmentation:", batch_length_after)

/Users/andreeaioanatudor/Desktop/Uni/Msc/HWR/Handwriting-Recognition/english_handwriting/venv/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17768592637663142582


<class 'tensorflow.python.framework.ops.Tensor'>


AttributeError: in user code:

    File "/var/folders/3d/1fsp6_k10ng99vqrbkbg8s100000gn/T/ipykernel_55951/644587297.py", line 14, in None  *
        lambda x: {'image': augment_img(x['image']), 'label': x['label']}
    File "/var/folders/3d/1fsp6_k10ng99vqrbkbg8s100000gn/T/ipykernel_55951/2922131686.py", line 15, in augment_img  *
        img = tf.make_ndarray(img)

    AttributeError: 'Tensor' object has no attribute 'tensor_shape'


In [63]:
decoder_vocab[1] = ' '
decoder = StringLookup(vocabulary=decoder_vocab,
                           mask_token=None, invert=True)

In [68]:
for data in augmented_images_batches.take(1):
    # batch_length = tf.data.experimental.cardinality(
    #     data).numpy()
    # print(batch_length)

    images, labels = data['image'], data['label']

    for i in range(4):
        # print(decoder_vocab)
        image = images[i]
        label = labels[i]
        image = tf.keras.preprocessing.image.array_to_img(image)

        print(label)
        # Display the image using Matplotlib
        plt.imshow(image, cmap='gray')
        plt.axis('off')


        indices = tf.gather(label, tf.where(tf.math.not_equal(label, 99)))
        # Convert to string.
        label = tf.strings.reduce_join(decoder(indices))
        label = label.numpy().decode("utf-8")
        plt.title(label)
        plt.show()

NameError: name 'augmented_images_batches' is not defined